In [1]:
# NB: This is for spark running on parquet files converted from
#     baconbits skims. This is a prototype, there is lots of boilerplate.
#     We're making it better :-)

import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[1]') \
#    .master('spark://cmsspark-submit.fnal.gov:7077') \
conf = pyspark.sql.SparkSession.builder \
    .appName('decaf-%s' % guid()) \
    .master('spark://cmsspark-submit.fnal.gov:7077') \
    .config('spark.driver.cores',8) \
    .config('spark.driver.memory','10g') \
    .config('spark.executor.memory', "20g") \
    .config('spark.debug.maxToStringFields',1000) \
    .config('spark.executorEnv.XRD_RUNFORKHANDLER','1') \
    .config('spark.executorEnv.X509_PROXY',"%s" % os.environ['X509_USER_PROXY']) \
    .config('spark.executorEnv.ARROW_PRE_0_15_IPC_FORMAT','1') \
    .config('spark.files','%s' % os.environ['X509_USER_PROXY']) \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config("spark.blockManager.port", "8000") \
    .config('spark.shuffle.service.enabled','true') \
    .config('spark.scheduler.mode', 'FAIR') \
    .config('spark.executor.instances','1') \
    .config('spark.dynamicAllocation.enabled','true') \
    .config('spark.dynamicAllocation.executorIdleTimeout','1m') \
    .config('spark.dynamicAllocation.cachedExecutorIdleTimeout','10m') \
    .config('spark.dynamicAllocation.minExecutors','1') \
    .config('spark.dynamicAllocation.maxExecutors','80') \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.hadoop.fs.xrootd.read.buffer', 100 * 1048576) \
    .config('spark.hadoop.fs.xrootd.write.buffer', 100 * 1048576) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='0.4.2-SNAPSHOT')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

#from sparkmeasure import TaskMetrics
#taskmetrics = TaskMetrics(spark)

partitionsize = 50000
thread_workers = 32


In [4]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

fileslice = slice(None)
with open("metadata/2018.json") as fin:
    samplefiles = json.load(fin)

filelist = {}
i = 0
for dataset, info in samplefiles.items():
    i = i + 1
    #print(i)
    #if i > 10: continue
    files = []
    for file in info['files'][fileslice]:
        files.append(file)
        filelist[dataset] = {'files': files, 'treename': 'Events'}
        #if 'MET____0_' in dataset: 
        #    filelist[dataset] = {'files': files, 'treename': 'Events'}
#print(filelist['MET____0_'])

print(filelist[dataset])

{'files': ['root://cmsxrootd.fnal.gov//store/user/jongho/DarkHiggs/nanoAOD/nJetMax1/MonoZ_Mzprime_500_Mchi_150_2018/MonoZ_MZprime_500_Mchi_150/NanoTuples-2018_MonoZ_MZprime_500_Mchi_150/191001_190301/0000/nano_1.root', 'root://cmsxrootd.fnal.gov//store/user/jongho/DarkHiggs/nanoAOD/nJetMax1/MonoZ_Mzprime_500_Mchi_150_2018/MonoZ_MZprime_500_Mchi_150/NanoTuples-2018_MonoZ_MZprime_500_Mchi_150/191001_190301/0000/nano_10.root', 'root://cmsxrootd.fnal.gov//store/user/jongho/DarkHiggs/nanoAOD/nJetMax1/MonoZ_Mzprime_500_Mchi_150_2018/MonoZ_MZprime_500_Mchi_150/NanoTuples-2018_MonoZ_MZprime_500_Mchi_150/191001_190301/0000/nano_100.root', 'root://cmsxrootd.fnal.gov//store/user/jongho/DarkHiggs/nanoAOD/nJetMax1/MonoZ_Mzprime_500_Mchi_150_2018/MonoZ_MZprime_500_Mchi_150/NanoTuples-2018_MonoZ_MZprime_500_Mchi_150/191001_190301/0000/nano_101.root', 'root://cmsxrootd.fnal.gov//store/user/jongho/DarkHiggs/nanoAOD/nJetMax1/MonoZ_Mzprime_500_Mchi_150_2018/MonoZ_MZprime_500_Mchi_150/NanoTuples-2018_Mono

In [5]:
#get the dark Higgs analysis worker
from coffea.util import load

processor_instance = load('processors/darkhiggs2018.coffea')


In [ ]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor
import pyspark.sql.functions as F

def presel_filter(df):
    df = df.filter(F.expr('exists(AK15Puppi_pt, x -> x > 160)'))
    return df

tic = time.time()
final_accumulator = run_spark_job(filelist, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                  executor_args={'file_type': 'root', 'cache': False, 'df_xform':presel_filter})
dt = time.time() - tic


/usr/local/lib/python3.6/site-packages/coffea/processor/executor.py:715: UserWarning: If you are using pyarrow >= 0.15.0, make sure to set ARROW_PRE_0_15_IPC_FORMAT=1 in your environment!
  warnings.warn('If you are using pyarrow >= 0.15.0, make sure to set %s=%s in your environment!' % arrow_env)


In [7]:
nevt = sum(spark_executor.counts.values())
print('processed:',nevt,'events')
print('total time: ',dt/60)
print('μs/evt', dt/nevt*1e6)
print('Mevt/s', nevt/dt/1e6)


processed: 2610796968 events
total time:  103.61794255574544
μs/evt 2.3812945355560586
Mevt/s 0.4199396526001303


In [ ]:
from coffea import hist
from coffea.util import save
import numpy as np

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
print("Processed %.1fM events" % (nevt/1e6, ))
print("Filled %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

# Pickle is not very fast or memory efficient, will be replaced by something better soon
save(final_accumulator,'pods/pods2018.coffea')

#dt = time.time() - tstart
#print("%.2f us*cpu/event overall" % (1e6*dt*nworkers/final_accumulators['nentries'], ))


In [8]:
spark.stop()
